In [1]:
# Cài đặt các thư viện cần thiết để chạy chương trình
%tensorflow_version 1.x
!pip install pyvi
!pip install ftfy
!pip install gensim

TensorFlow 1.x selected.
     |████████████████████████████████| 8.5MB 3.7MB/s 
     |████████████████████████████████| 747kB 55.3MB/s 
     |████████████████████████████████| 71kB 3.3MB/s 
  Created wheel for ftfy: filename=ftfy-5.8-cp36-none-any.whl size=45612 sha256=8a1b3a42b1d6b0cd020ee0d04d3560ba0486005d53139ddf3e22b985d89ca471
  Stored in directory: /root/.cache/pip/wheels/ba/c0/ef/f28c4da5ac84a4e06ac256ca9182fc34fa57fefffdbc68425b
Successfully built ftfy


In [2]:
# -*- coding: utf-8 -*-
import numpy as np
import tensorflow as tf
import os, pickle, re, keras, sklearn, string
from keras.callbacks import *
from keras.preprocessing.text import *
from keras.preprocessing.sequence import *
from keras.models import load_model
from pyvi import ViTokenizer, ViPosTagger
from keras.models import Model
from keras.layers import *
from keras import optimizers
from keras.utils.vis_utils import plot_model
import operator

Using TensorFlow backend.


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
path_train ='/content/gdrive/My Drive/ABSA/Dataset/Train.txt'
path_dev ='/content/gdrive/My Drive/ABSA/Dataset/Dev.txt'
path_test ='/content/gdrive/My Drive/ABSA/Dataset/Test.txt'


Mounted at /content/gdrive


In [4]:
def normalText(sent):
    sent = str(sent).replace('_',' ').replace('/',' trên ')
    sent = re.sub('-{2,}','',sent)
    sent = re.sub('\\s+',' ', sent)
    patPrice = r'([0-9]+k?(\s?-\s?)[0-9]+\s?(k|K))|([0-9]+(.|,)?[0-9]+\s?(triệu|ngàn|trăm|k|K|))|([0-9]+(.[0-9]+)?Ä‘)|([0-9]+k)'
    patURL = r"(?:http://|www.)[^\"]+"
    sent = re.sub(patURL,'website',sent)
    sent = re.sub(patPrice, ' giá_tiền ', sent)
    sent = re.sub('\.+','.',sent)
    sent = re.sub('(hagtag\\s+)+',' hagtag ',sent)
    sent = re.sub('\\s+',' ',sent)
    return sent

def deleteIcon(text):
    text = text.lower()
    s = ''
    pattern = r"[a-zA-ZaăâbcdđeêghiklmnoôơpqrstuưvxyàằầbcdđèềghìklmnòồờpqrstùừvxỳáắấbcdđéếghíklmnóốớpqrstúứvxýảẳẩbcdđẻểghỉklmnỏổởpqrstủửvxỷạặậbcdđẹệghịklmnọộợpqrstụựvxỵãẵẫbcdđẽễghĩklmnõỗỡpqrstũữvxỹAĂÂBCDĐEÊGHIKLMNOÔƠPQRSTUƯVXYÀẰẦBCDĐÈỀGHÌKLMNÒỒỜPQRSTÙỪVXỲÁẮẤBCDĐÉẾGHÍKLMNÓỐỚPQRSTÚỨVXÝẠẶẬBCDĐẸỆGHỊKLMNỌỘỢPQRSTỤỰVXỴẢẲẨBCDĐẺỂGHỈKLMNỎỔỞPQRSTỦỬVXỶÃẴẪBCDĐẼỄGHĨKLMNÕỖỠPQRSTŨỮVXỸ,._]"
    
    for char in text:
        if char !=' ':
            if len(re.findall(pattern, char)) != 0:
                s+=char
            elif char == '_':
                s+=char
        else:
            s+=char
    s = re.sub('\\s+',' ',s)
    return s.strip()


def clean_doc(doc):
    for punc in string.punctuation:
        doc = doc.replace(punc,' '+ punc + ' ')
    doc = normalText(doc)
    doc = deleteIcon(doc)
    doc = ViTokenizer.tokenize(doc)
    # Lowercase
    doc = doc.lower()
    # Removing multiple whitespaces
    doc = re.sub(r"\?", " \? ", doc)
    # Remove numbers
    doc = re.sub(r"[0-9]+", " num ", doc)
    # Split in tokens
    # Remove punctuation
    for punc in string.punctuation:
      if punc not in "_":
          doc = doc.replace(punc,' ')
    doc = re.sub('\\s+',' ',doc)
    return doc

In [7]:
from keras.models import load_model
path_save ='/content/gdrive/My Drive/ABSA/'
model = load_model(path_save + 'BiLSTM_CNN_model.h5')

with open('/content/gdrive/My Drive/ABSA/input_tokenizer.pkl', 'rb') as fp:
    input_tokenizer = pickle.load(fp)

maxLength = 548

In [9]:
listLabel = 'FOOD#STYLE&OPTIONS,FOOD#PRICES,FOOD#QUALITY,DRINKS#STYLE&OPTIONS,DRINKS#QUALITY,DRINKS#PRICES,RESTAURANT#GENERAL,RESTAURANT#MISCELLANEOUS,RESTAURANT#PRICES,LOCATION#GENERAL,SERVICE#GENERAL,AMBIENCE#GENERAL'
categories = listLabel.split(',')

In [ ]:
# Hàm dự đoán trên 1 dữ liệu đầu vào
input_text = "Món ăn ngon và nhân viên phục vụ rất tích cực"
input_clean = clean_doc(input_text)
input_index = np.array(pad_sequences(input_tokenizer.texts_to_sequences([input_clean]), maxlen=maxLength,padding="post"))

predicted = model.predict([np.expand_dims(input_index[0], axis=0)])
output = ''
for i, predict in enumerate(predicted):
    index2, value = max(enumerate(predict[0]), key=operator.itemgetter(1))
    if index2 == 1:
        output+= '{' + str(categories[i]) + ', positive}, '
    elif index2 == 2:
        output+= '{' + str(categories[i]) + ', neutral}, '
    elif index2 == 3:
        output+= '{' + str(categories[i]) + ', negative}, '
output = output[:-2]
print("Đầu vào: ", input_text)
print("Dự đoán: ", output)